In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
os.environ["groq_api_key"] = os.getenv("groq_api_key")

In [15]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen-2.5-32b")

In [16]:
llm.invoke("hii")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 31, 'total_tokens': 41, 'completion_time': 0.026388079, 'prompt_time': 0.003920304, 'queue_time': 0.020915141, 'total_time': 0.030308383}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-2012865f-c12b-47b8-8b67-5a0e8f300745-0', usage_metadata={'input_tokens': 31, 'output_tokens': 10, 'total_tokens': 41})

In [18]:
from langchain_community.document_loaders import PyPDFLoader

In [27]:
file_path = r"C:\Users\ROHIT\OneDrive\Desktop\PN\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf"
loader = PyPDFLoader(file_path)
docs=loader.load()
len(docs)

2

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
splits=text_splitter.split_documents(docs)

In [36]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [40]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

ModuleNotFoundError: No module named 'langchain_qdrant'